# Granite 3.1 8B Deployment Verification
## Module 3: Generative Model - Deployment Testing

---

**Objective:** Verify that Granite 3.1 8B Instruct is properly deployed and serving requests through vLLM

**What this notebook validates:**
- ✅ InferenceService deployment status
- ✅ API endpoint accessibility
- ✅ Model loading completion
- ✅ Text generation capabilities
- ✅ Response format validation
- ✅ Performance baseline measurements

---

## 📋 Step 1: Import Required Libraries

In [ ]:
import os
import time
import json
import requests
import subprocess
from datetime import datetime
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

# Try importing OpenAI client (should be installed from previous modules)
try:
    import openai
    print("✅ OpenAI client available")
except ImportError:
    print("⚠️  Installing OpenAI client...")
    !pip install openai
    import openai
    print("✅ OpenAI client installed and imported")

print(f"📅 Verification started: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

## 🔍 Step 2: Check InferenceService Status

In [ ]:
def check_inferenceservice_status():
    """
    Check the status of the Granite 3.1 8B InferenceService
    """
    print("🔍 Checking InferenceService status...")
    
    service_name = "granite-3-1-8b-instruct"
    
    try:
        # Get InferenceService status
        result = subprocess.run(
            ["oc", "get", "inferenceservice", service_name, "-o", "json"],
            capture_output=True,
            text=True,
            timeout=30
        )
        
        if result.returncode != 0:
            print(f"❌ Failed to get InferenceService status: {result.stderr}")
            return False, None
        
        service_info = json.loads(result.stdout)
        
        # Extract key information
        metadata = service_info.get('metadata', {})
        status = service_info.get('status', {})
        
        print(f"📊 Service Name: {metadata.get('name', 'Unknown')}")
        print(f"📊 Namespace: {metadata.get('namespace', 'Unknown')}")
        print(f"📊 Creation Time: {metadata.get('creationTimestamp', 'Unknown')}")
        
        # Check conditions
        conditions = status.get('conditions', [])
        
        ready_condition = None
        for condition in conditions:
            condition_type = condition.get('type')
            condition_status = condition.get('status')
            last_transition = condition.get('lastTransitionTime', '')
            reason = condition.get('reason', '')
            message = condition.get('message', '')
            
            print(f"\n📋 Condition: {condition_type}")
            print(f"   Status: {condition_status}")
            print(f"   Reason: {reason}")
            print(f"   Message: {message[:100]}{'...' if len(message) > 100 else ''}")
            print(f"   Last Transition: {last_transition}")
            
            if condition_type == 'Ready':
                ready_condition = condition
        
        # Check if service is ready
        is_ready = ready_condition and ready_condition.get('status') == 'True'
        
        # Get URL if available
        url = status.get('url', 'Not available')
        
        print(f"\n📡 Service URL: {url}")
        
        if is_ready:
            print(f"\n✅ InferenceService is READY!")
            return True, url
        else:
            print(f"\n⚠️  InferenceService is NOT ready yet")
            print(f"   This is normal during initial deployment (model loading can take 5-10 minutes)")
            return False, url
            
    except subprocess.TimeoutExpired:
        print(f"❌ Timeout while checking InferenceService status")
        return False, None
    except json.JSONDecodeError as e:
        print(f"❌ Failed to parse InferenceService JSON: {e}")
        return False, None
    except Exception as e:
        print(f"❌ Error checking InferenceService: {e}")
        return False, None

# Check the service status
is_ready, service_url = check_inferenceservice_status()

## 🔗 Step 3: Check Pod Status and Logs

In [ ]:
def check_pod_status():
    """
    Check the status of pods related to the Granite model
    """
    print("🔍 Checking pod status...")
    
    try:
        # Get pods with granite label
        result = subprocess.run(
            ["oc", "get", "pods", "-l", "serving.kserve.io/inferenceservice=granite-3-1-8b-instruct", "-o", "json"],
            capture_output=True,
            text=True,
            timeout=30
        )
        
        if result.returncode != 0:
            print(f"❌ Failed to get pod status: {result.stderr}")
            return False
        
        pods_info = json.loads(result.stdout)
        pods = pods_info.get('items', [])
        
        if not pods:
            print(f"⚠️  No pods found for granite-3-1-8b-instruct")
            print(f"   This might indicate the deployment hasn't started yet")
            return False
        
        print(f"📊 Found {len(pods)} pod(s):")
        
        pod_ready = False
        
        for i, pod in enumerate(pods):
            metadata = pod.get('metadata', {})
            status = pod.get('status', {})
            
            pod_name = metadata.get('name', f'pod-{i}')
            pod_phase = status.get('phase', 'Unknown')
            creation_time = metadata.get('creationTimestamp', 'Unknown')
            
            print(f"\n📦 Pod {i+1}: {pod_name}")
            print(f"   Phase: {pod_phase}")
            print(f"   Created: {creation_time}")
            
            # Check container statuses
            container_statuses = status.get('containerStatuses', [])
            
            for container in container_statuses:
                container_name = container.get('name', 'unknown')
                container_ready = container.get('ready', False)
                restart_count = container.get('restartCount', 0)
                
                print(f"   Container '{container_name}': {'Ready' if container_ready else 'Not Ready'}")
                print(f"   Restart count: {restart_count}")
                
                # Check container state
                state = container.get('state', {})
                if 'running' in state:
                    started_at = state['running'].get('startedAt', 'Unknown')
                    print(f"   Running since: {started_at}")
                    if container_ready:
                        pod_ready = True
                elif 'waiting' in state:
                    reason = state['waiting'].get('reason', 'Unknown')
                    message = state['waiting'].get('message', '')
                    print(f"   Waiting: {reason}")
                    if message:
                        print(f"   Message: {message[:100]}{'...' if len(message) > 100 else ''}")
                elif 'terminated' in state:
                    reason = state['terminated'].get('reason', 'Unknown')
                    exit_code = state['terminated'].get('exitCode', 'Unknown')
                    print(f"   Terminated: {reason} (exit code: {exit_code})")
        
        return pod_ready
        
    except Exception as e:
        print(f"❌ Error checking pod status: {e}")
        return False

def get_recent_logs():
    """
    Get recent logs from the granite model pod
    """
    print("\n📋 Getting recent logs...")
    
    try:
        # Get the pod name
        result = subprocess.run(
            ["oc", "get", "pods", "-l", "serving.kserve.io/inferenceservice=granite-3-1-8b-instruct", "-o", "jsonpath={.items[0].metadata.name}"],
            capture_output=True,
            text=True,
            timeout=30
        )
        
        if result.returncode != 0 or not result.stdout.strip():
            print(f"⚠️  Could not find pod name")
            return
        
        pod_name = result.stdout.strip()
        print(f"📦 Getting logs from pod: {pod_name}")
        
        # Get logs (last 20 lines)
        result = subprocess.run(
            ["oc", "logs", pod_name, "--tail=20"],
            capture_output=True,
            text=True,
            timeout=30
        )
        
        if result.returncode == 0:
            logs = result.stdout.strip()
            if logs:
                print(f"\n📄 Recent logs:")
                print("-" * 80)
                print(logs)
                print("-" * 80)
                
                # Check for key indicators in logs
                if "Model loaded successfully" in logs or "Uvicorn running" in logs:
                    print("✅ Model appears to be loaded and serving!")
                elif "Loading model" in logs or "Downloading" in logs:
                    print("🔄 Model is still loading...")
                elif "Error" in logs or "Failed" in logs:
                    print("⚠️  Potential issues detected in logs")
            else:
                print("📄 No recent logs available")
        else:
            print(f"❌ Failed to get logs: {result.stderr}")
            
    except Exception as e:
        print(f"❌ Error getting logs: {e}")

# Check pod status and logs
pod_ready = check_pod_status()
get_recent_logs()

## 🌐 Step 4: Test API Endpoint Accessibility

In [ ]:
def determine_endpoint_url():
    """
    Determine the correct endpoint URL for the Granite model
    """
    print("🔍 Determining endpoint URL...")
    
    # Try to get the service URL from InferenceService
    if service_url and service_url != 'Not available':
        # External URL format
        external_url = f"{service_url}/v1"
        print(f"📡 External URL: {external_url}")
        return external_url
    
    # Fallback to internal service URL
    try:
        # Get current namespace
        result = subprocess.run(
            ["oc", "project", "-q"],
            capture_output=True,
            text=True,
            timeout=10
        )
        
        namespace = result.stdout.strip() if result.returncode == 0 else "default"
        internal_url = f"http://granite-3-1-8b-instruct-predictor.{namespace}.svc.cluster.local:8080/v1"
        
        print(f"📡 Internal URL: {internal_url}")
        return internal_url
        
    except Exception as e:
        print(f"⚠️  Could not determine namespace: {e}")
        fallback_url = "http://granite-3-1-8b-instruct-predictor:8080/v1"
        print(f"📡 Fallback URL: {fallback_url}")
        return fallback_url

def test_endpoint_connectivity(endpoint_url, timeout=30):
    """
    Test basic connectivity to the model endpoint
    """
    print(f"\n🔗 Testing endpoint connectivity: {endpoint_url}")
    
    # Test basic HTTP connectivity
    try:
        # Try to access the root endpoint
        response = requests.get(
            endpoint_url.replace('/v1', ''),
            timeout=timeout
        )
        
        print(f"📊 HTTP Status Code: {response.status_code}")
        print(f"📊 Response Headers: {dict(response.headers)}")
        
        if response.status_code in [200, 404]:  # 404 is OK for root endpoint
            print(f"✅ Endpoint is accessible")
            return True
        else:
            print(f"⚠️  Unexpected status code: {response.status_code}")
            return False
            
    except requests.exceptions.ConnectTimeout:
        print(f"❌ Connection timeout - endpoint may not be ready")
        return False
    except requests.exceptions.ConnectionError as e:
        print(f"❌ Connection error: {e}")
        print(f"   This usually means the service is not yet ready")
        return False
    except Exception as e:
        print(f"❌ Error testing connectivity: {e}")
        return False

def test_openai_api_compatibility(endpoint_url):
    """
    Test if the endpoint is compatible with OpenAI API format
    """
    print(f"\n🔍 Testing OpenAI API compatibility...")
    
    try:
        # Test the /v1/models endpoint
        models_url = f"{endpoint_url}/models"
        response = requests.get(models_url, timeout=10)
        
        print(f"📊 Models endpoint status: {response.status_code}")
        
        if response.status_code == 200:
            models_data = response.json()
            print(f"✅ OpenAI API format supported")
            print(f"📊 Available models: {[model.get('id', 'unknown') for model in models_data.get('data', [])]}")
            return True
        else:
            print(f"⚠️  Models endpoint returned: {response.status_code}")
            return False
            
    except Exception as e:
        print(f"⚠️  Could not test OpenAI API compatibility: {e}")
        return False

# Determine and test endpoint
endpoint_url = determine_endpoint_url()
connectivity_ok = test_endpoint_connectivity(endpoint_url)
api_compatible = test_openai_api_compatibility(endpoint_url) if connectivity_ok else False

## 🧪 Step 5: Test Text Generation Capabilities

In [ ]:
def test_text_generation(endpoint_url):
    """
    Test basic text generation capabilities
    """
    print("🧪 Testing text generation capabilities...")
    
    if not connectivity_ok:
        print("❌ Skipping generation test - endpoint not accessible")
        return False
    
    try:
        # Configure OpenAI client for vLLM endpoint
        client = openai.OpenAI(
            base_url=endpoint_url,
            api_key="not-used"  # vLLM doesn't require API key
        )
        
        # Test cases with different types of prompts
        test_cases = [
            {
                "name": "Simple Completion",
                "prompt": "The capital of France is",
                "max_tokens": 10,
                "temperature": 0.1
            },
            {
                "name": "Product Description",
                "prompt": "Write a brief product description for wireless bluetooth headphones:",
                "max_tokens": 50,
                "temperature": 0.7
            },
            {
                "name": "Creative Writing",
                "prompt": "Once upon a time in a digital marketplace,",
                "max_tokens": 30,
                "temperature": 0.8
            }
        ]
        
        generation_results = []
        
        for i, test_case in enumerate(test_cases):
            print(f"\n📝 Test {i+1}: {test_case['name']}")
            print(f"   Prompt: \"{test_case['prompt']}\"")
            
            try:
                start_time = time.time()
                
                # Make the generation request
                response = client.completions.create(
                    model="granite-3-1-8b",  # This should match the served model name
                    prompt=test_case['prompt'],
                    max_tokens=test_case['max_tokens'],
                    temperature=test_case['temperature'],
                    stop=["\n\n"]
                )
                
                end_time = time.time()
                response_time = end_time - start_time
                
                if response.choices and len(response.choices) > 0:
                    generated_text = response.choices[0].text.strip()
                    
                    print(f"   ✅ Generated: \"{generated_text}\"")
                    print(f"   ⏱️  Response time: {response_time:.2f} seconds")
                    
                    # Check token usage if available
                    if hasattr(response, 'usage') and response.usage:
                        print(f"   📊 Tokens used: {response.usage.total_tokens}")
                    
                    generation_results.append({
                        'test_name': test_case['name'],
                        'success': True,
                        'response_time': response_time,
                        'generated_length': len(generated_text),
                        'prompt_length': len(test_case['prompt'])
                    })
                    
                else:
                    print(f"   ❌ No text generated")
                    generation_results.append({
                        'test_name': test_case['name'],
                        'success': False,
                        'error': 'No choices in response'
                    })
                
            except Exception as e:
                print(f"   ❌ Generation failed: {e}")
                generation_results.append({
                    'test_name': test_case['name'],
                    'success': False,
                    'error': str(e)
                })
        
        # Summary of results
        successful_tests = sum(1 for result in generation_results if result['success'])
        total_tests = len(generation_results)
        
        print(f"\n📊 Generation Test Summary:")
        print(f"   Successful tests: {successful_tests}/{total_tests}")
        
        if successful_tests > 0:
            avg_response_time = sum(r['response_time'] for r in generation_results if r['success']) / successful_tests
            print(f"   Average response time: {avg_response_time:.2f} seconds")
        
        return successful_tests == total_tests
        
    except Exception as e:
        print(f"❌ Failed to test text generation: {e}")
        return False

# Test text generation
generation_ok = test_text_generation(endpoint_url)

## ⚡ Step 6: Performance Baseline Testing

In [ ]:
def run_performance_baseline(endpoint_url, num_requests=5):
    """
    Run basic performance baseline tests
    """
    print(f"⚡ Running performance baseline ({num_requests} requests)...")
    
    if not generation_ok:
        print("❌ Skipping performance test - generation not working")
        return False
    
    try:
        client = openai.OpenAI(
            base_url=endpoint_url,
            api_key="not-used"
        )
        
        # Standard prompt for consistency
        test_prompt = "Generate a product description for a smart watch with fitness tracking features:"
        
        response_times = []
        token_counts = []
        success_count = 0
        
        print(f"\n🔄 Running {num_requests} sequential requests...")
        
        for i in range(num_requests):
            try:
                start_time = time.time()
                
                response = client.completions.create(
                    model="granite-3-1-8b",
                    prompt=test_prompt,
                    max_tokens=100,
                    temperature=0.7
                )
                
                end_time = time.time()
                response_time = end_time - start_time
                
                if response.choices and len(response.choices) > 0:
                    generated_text = response.choices[0].text
                    token_count = len(generated_text.split())  # Rough token estimate
                    
                    response_times.append(response_time)
                    token_counts.append(token_count)
                    success_count += 1
                    
                    print(f"   Request {i+1}: {response_time:.2f}s, ~{token_count} tokens")
                else:
                    print(f"   Request {i+1}: Failed - no response")
                
            except Exception as e:
                print(f"   Request {i+1}: Failed - {e}")
        
        # Calculate performance metrics
        if response_times:
            avg_response_time = sum(response_times) / len(response_times)
            min_response_time = min(response_times)
            max_response_time = max(response_times)
            
            avg_tokens = sum(token_counts) / len(token_counts) if token_counts else 0
            tokens_per_second = avg_tokens / avg_response_time if avg_response_time > 0 else 0
            
            print(f"\n📊 Performance Baseline Results:")
            print(f"   Success rate: {success_count}/{num_requests} ({success_count/num_requests*100:.1f}%)")
            print(f"   Average response time: {avg_response_time:.2f} seconds")
            print(f"   Min response time: {min_response_time:.2f} seconds")
            print(f"   Max response time: {max_response_time:.2f} seconds")
            print(f"   Average tokens generated: {avg_tokens:.1f}")
            print(f"   Estimated tokens/second: {tokens_per_second:.1f}")
            
            # Performance assessment
            if avg_response_time < 2.0:
                print(f"   ✅ Response time: Excellent (< 2s)")
            elif avg_response_time < 5.0:
                print(f"   ✅ Response time: Good (< 5s)")
            else:
                print(f"   ⚠️  Response time: Slow (> 5s)")
            
            if tokens_per_second > 30:
                print(f"   ✅ Token throughput: Excellent (> 30 tokens/s)")
            elif tokens_per_second > 15:
                print(f"   ✅ Token throughput: Good (> 15 tokens/s)")
            else:
                print(f"   ⚠️  Token throughput: Low (< 15 tokens/s)")
            
            return True
        else:
            print(f"❌ No successful requests for performance analysis")
            return False
            
    except Exception as e:
        print(f"❌ Performance baseline failed: {e}")
        return False

# Run performance baseline
performance_ok = run_performance_baseline(endpoint_url)

## 🎯 Step 7: E-commerce Specific Testing

In [ ]:
def test_ecommerce_use_cases(endpoint_url):
    """
    Test specific e-commerce use cases for the workshop
    """
    print("🛍️ Testing e-commerce specific use cases...")
    
    if not generation_ok:
        print("❌ Skipping e-commerce tests - generation not working")
        return False
    
    try:
        client = openai.OpenAI(
            base_url=endpoint_url,
            api_key="not-used"
        )
        
        # E-commerce specific test cases
        ecommerce_tests = [
            {
                "name": "Product Description Generation",
                "prompt": "Write a compelling product description for a wireless bluetooth headphone with noise cancellation:\n\nProduct: Sony WH-1000XM5\nFeatures: Active noise cancellation, 30-hour battery, lightweight design\n\nDescription:",
                "max_tokens": 80,
                "expected_keywords": ["noise", "battery", "wireless", "quality"]
            },
            {
                "name": "Customer Recommendation",
                "prompt": "Based on a customer's purchase history of electronics and fitness gear, recommend 3 products that would interest them:",
                "max_tokens": 100,
                "expected_keywords": ["recommend", "product", "customer"]
            },
            {
                "name": "Marketing Copy",
                "prompt": "Create engaging marketing copy for a summer sale on outdoor sports equipment. Include a call-to-action:",
                "max_tokens": 60,
                "expected_keywords": ["sale", "summer", "outdoor"]
            },
            {
                "name": "Product Comparison",
                "prompt": "Compare two smartphones focusing on camera quality and battery life:\n\nPhone A: 48MP camera, 4000mAh battery\nPhone B: 64MP camera, 3500mAh battery\n\nComparison:",
                "max_tokens": 70,
                "expected_keywords": ["camera", "battery", "better", "quality"]
            }
        ]
        
        successful_tests = 0
        
        for i, test in enumerate(ecommerce_tests):
            print(f"\n🧪 E-commerce Test {i+1}: {test['name']}")
            
            try:
                start_time = time.time()
                
                response = client.completions.create(
                    model="granite-3-1-8b",
                    prompt=test['prompt'],
                    max_tokens=test['max_tokens'],
                    temperature=0.7,
                    stop=["\n\n"]
                )
                
                end_time = time.time()
                
                if response.choices and len(response.choices) > 0:
                    generated_text = response.choices[0].text.strip()
                    
                    print(f"   ✅ Generated ({end_time - start_time:.2f}s):")
                    print(f"   \"{generated_text[:150]}{'...' if len(generated_text) > 150 else ''}\"")
                    
                    # Check for expected keywords (basic relevance check)
                    found_keywords = []
                    for keyword in test['expected_keywords']:
                        if keyword.lower() in generated_text.lower():
                            found_keywords.append(keyword)
                    
                    if found_keywords:
                        print(f"   📊 Relevant keywords found: {found_keywords}")
                        successful_tests += 1
                    else:
                        print(f"   ⚠️  No expected keywords found (might still be valid)")
                        successful_tests += 1  # Still count as success if text was generated
                    
                else:
                    print(f"   ❌ No text generated")
                    
            except Exception as e:
                print(f"   ❌ Test failed: {e}")
        
        print(f"\n📊 E-commerce Test Summary:")
        print(f"   Successful tests: {successful_tests}/{len(ecommerce_tests)}")
        
        return successful_tests >= len(ecommerce_tests) * 0.8  # 80% success rate
        
    except Exception as e:
        print(f"❌ E-commerce testing failed: {e}")
        return False

# Test e-commerce use cases
ecommerce_ok = test_ecommerce_use_cases(endpoint_url)

## 📊 Step 8: Resource Usage Check

In [ ]:
def check_resource_usage():
    """
    Check resource usage of the Granite model deployment
    """
    print("📊 Checking resource usage...")
    
    try:
        # Get pod resource usage
        result = subprocess.run(
            ["oc", "top", "pods", "-l", "serving.kserve.io/inferenceservice=granite-3-1-8b-instruct"],
            capture_output=True,
            text=True,
            timeout=30
        )
        
        if result.returncode == 0 and result.stdout.strip():
            print(f"📈 Current resource usage:")
            print(result.stdout)
            
            # Parse the output for analysis
            lines = result.stdout.strip().split('\n')
            if len(lines) > 1:  # Skip header
                for line in lines[1:]:
                    parts = line.split()
                    if len(parts) >= 3:
                        pod_name = parts[0]
                        cpu_usage = parts[1]
                        memory_usage = parts[2]
                        
                        print(f"\n📦 Pod: {pod_name}")
                        print(f"   CPU: {cpu_usage}")
                        print(f"   Memory: {memory_usage}")
        else:
            print(f"⚠️  Could not get resource usage: {result.stderr}")
            print(f"   This might be due to metrics server not being available")
        
        # Try to get resource limits from the deployment
        result = subprocess.run(
            ["oc", "get", "pods", "-l", "serving.kserve.io/inferenceservice=granite-3-1-8b-instruct", "-o", "jsonpath={.items[0].spec.containers[0].resources}"],
            capture_output=True,
            text=True,
            timeout=30
        )
        
        if result.returncode == 0 and result.stdout.strip():
            try:
                resources = json.loads(result.stdout)
                print(f"\n📋 Resource Configuration:")
                
                if 'requests' in resources:
                    print(f"   Requests: {resources['requests']}")
                if 'limits' in resources:
                    print(f"   Limits: {resources['limits']}")
                    
            except json.JSONDecodeError:
                print(f"   Raw resource spec: {result.stdout}")
        
        return True
        
    except Exception as e:
        print(f"❌ Error checking resource usage: {e}")
        return False

# Check resource usage
resource_check_ok = check_resource_usage()

## 📋 Step 9: Final Deployment Verification Summary

In [ ]:
def generate_verification_report():
    """
    Generate a comprehensive verification report
    """
    print("📋 GRANITE 3.1 8B DEPLOYMENT VERIFICATION REPORT")
    print("=" * 70)
    print(f"📅 Verification completed: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    print(f"🔗 Endpoint URL: {endpoint_url}")
    
    # Verification checklist
    verification_items = [
        ("InferenceService Status", "✅ Ready" if is_ready else "❌ Not Ready"),
        ("Pod Status", "✅ Running" if pod_ready else "❌ Not Running"),
        ("Endpoint Connectivity", "✅ Accessible" if connectivity_ok else "❌ Not Accessible"),
        ("OpenAI API Compatibility", "✅ Compatible" if api_compatible else "❌ Not Compatible"),
        ("Text Generation", "✅ Working" if generation_ok else "❌ Not Working"),
        ("Performance Baseline", "✅ Passed" if performance_ok else "❌ Failed"),
        ("E-commerce Use Cases", "✅ Passed" if ecommerce_ok else "❌ Failed"),
        ("Resource Monitoring", "✅ Available" if resource_check_ok else "⚠️  Limited")
    ]
    
    print(f"\n📊 VERIFICATION CHECKLIST:")
    print("-" * 50)
    
    passed_checks = 0
    total_checks = len(verification_items)
    
    for item, status in verification_items:
        print(f"   {status:<15} {item}")
        if "✅" in status:
            passed_checks += 1
    
    print("-" * 50)
    print(f"   Overall Score: {passed_checks}/{total_checks} ({passed_checks/total_checks*100:.0f}%)")
    
    # Overall assessment
    if passed_checks >= total_checks * 0.9:  # 90% pass rate
        print(f"\n🎉 DEPLOYMENT VERIFICATION SUCCESSFUL!")
        print(f"\n✅ Your Granite 3.1 8B model is ready for production use!")
        print(f"\n📚 Ready to proceed to:")
        print(f"   📂 Module 4: LangChain Integration")
        print(f"   📄 File: 04-langchain-integration.md")
        
        print(f"\n💡 What's next:")
        print(f"   • Create LangChain wrappers for both models")
        print(f"   • Build e-commerce AI workflows")
        print(f"   • Develop interactive dashboard")
        print(f"   • Test end-to-end system integration")
        
    elif passed_checks >= total_checks * 0.7:  # 70% pass rate
        print(f"\n⚠️  DEPLOYMENT PARTIALLY READY")
        print(f"   The model is deployed but some features may not work optimally")
        print(f"   You can proceed with caution or troubleshoot the failed checks")
        
    else:
        print(f"\n❌ DEPLOYMENT VERIFICATION FAILED")
        print(f"   Too many critical issues detected")
        print(f"   Please troubleshoot before proceeding to the next module")
    
    # Troubleshooting guidance
    if not is_ready or not pod_ready:
        print(f"\n🔧 TROUBLESHOOTING TIPS:")
        print(f"   • Model loading can take 5-10 minutes on first deployment")
        print(f"   • Check pod logs: oc logs <pod-name> -f")
        print(f"   • Verify GPU resources are available")
        print(f"   • Check if the model download completed successfully")
    
    if not connectivity_ok or not generation_ok:
        print(f"\n🔧 API TROUBLESHOOTING:")
        print(f"   • Verify service is running: oc get svc")
        print(f"   • Check network policies and firewall rules")
        print(f"   • Test with port-forward: oc port-forward svc/granite-3-1-8b-instruct 8080:8080")
    
    print(f"\n📧 Need help? Contact: cestay@redhat.com")
    print(f"🐙 Workshop repo: https://github.com/pkstaz/ai-ecommerce-workshop")
    print("=" * 70)
    
    return passed_checks >= total_checks * 0.7

# Generate final verification report
verification_successful = generate_verification_report()

## 🔍 Step 10: Connection Configuration for Next Module

In [ ]:
def save_connection_config():
    """
    Save connection configuration for use in LangChain integration module
    """
    print("💾 Saving connection configuration for LangChain integration...")
    
    if not verification_successful:
        print("⚠️  Skipping config save - deployment verification failed")
        return
    
    try:
        # Create configuration for next module
        config = {
            "granite_endpoint": endpoint_url,
            "model_name": "granite-3-1-8b",
            "deployment_ready": verification_successful,
            "verification_timestamp": datetime.now().isoformat(),
            "service_name": "granite-3-1-8b-instruct",
            "api_format": "openai",
            "recommended_parameters": {
                "temperature": 0.7,
                "max_tokens": 150,
                "top_p": 0.9,
                "frequency_penalty": 0.1
            }
        }
        
        # Save to file
        config_file = "granite_connection_config.json"
        with open(config_file, 'w') as f:
            json.dump(config, f, indent=2)
        
        print(f"✅ Configuration saved to: {config_file}")
        print(f"📊 Configuration details:")
        for key, value in config.items():
            if key != "recommended_parameters":
                print(f"   {key}: {value}")
        
        # Environment variables for easy access
        print(f"\n🔧 Environment variables for LangChain:")
        print(f"   export GRANITE_ENDPOINT_URL='{endpoint_url}'")
        print(f"   export GRANITE_MODEL_NAME='granite-3-1-8b'")
        
    except Exception as e:
        print(f"❌ Error saving configuration: {e}")

# Save configuration if verification successful
save_connection_config()

---

## 📝 Summary

This notebook has comprehensively verified your Granite 3.1 8B deployment:

✅ **InferenceService Status** - Checked deployment readiness and health  
✅ **Pod and Resource Monitoring** - Verified container status and resource usage  
✅ **API Endpoint Testing** - Confirmed OpenAI-compatible API accessibility  
✅ **Text Generation Validation** - Tested core generation capabilities  
✅ **Performance Baseline** - Measured response times and throughput  
✅ **E-commerce Use Cases** - Validated specific workshop scenarios  
✅ **Connection Configuration** - Prepared settings for LangChain integration  

**Verification Results:**
- **Model Status:** Ready for production use
- **API Compatibility:** OpenAI format supported
- **Performance:** Meeting target metrics
- **Use Case Validation:** E-commerce scenarios working

**Ready for Module 4:** LangChain Integration

---